## Using Spark Cluster with shared data in Docker

### Handle Spark DataFrames

#### Install Java

Visite [https://www.java.com/](https://www.java.com/) to download and install Java.

#### Create a Python Virtual Environment

Select the Python interpreter to use for the notebook. Choose Python Environment from the drop-down menu and Create a Python Virtual Environment.

In [ ]:
!pip install pyspark==3.5.5 pandas pyarrow

#### Download csv file to local directory

In [ ]:
import urllib.request
import zipfile
from os import remove

url = 'https://www.kaggle.com/api/v1/datasets/download/chaitanyahivlekar/large-movie-dataset'
urllib.request.urlretrieve(url,'movies.zip')

with zipfile.ZipFile('movies.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

remove('movies.zip')

#### Connect to Spark and create Spark Session

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MyApp") \
    .master("local[*]") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()  


#### Reading data
##### Create a spark data frame from csv file

To create a spark data frame from a csv file, we can use the `read.csv` function. Using `inferSchema=True` allow spark infer correct type for each field. However, it can be slow.

In [ ]:
sdf = spark.read.csv("movies_dataset.csv", header=True, inferSchema=True)

#### Viewing Data

To see the schema, use `.printSchema()` method.

In [ ]:
sdf.printSchema()

Let's show first rows of spark data frame with method `.show()`

In [ ]:
sdf.show(1)

In [ ]:
sdf.show(truncate=False)

Display 5 first rows in list

In [ ]:
sdf.take(5)

Count total of rows.

In [ ]:
sdf.count()

View columns

In [ ]:
sdf.columns

Show a summary of spark dataframe

In [ ]:
sdf.describe().show()

#### View data with selected columns

In [ ]:
sdf.select("Movie_Name").show(5)

In [ ]:
sdf.select("Movie_Name", "Rating").show(truncate=False)

#### Filtering data

To filter data, we can use the `filter` method from spark data frame.


In [ ]:
from pyspark.sql.functions import col

sdf.filter(col("Rating") >= 5 ).show()

Here, we are using filter conditions `&` and `|` to filter the data.

In [ ]:


sdf.filter( (col("Rating") >= 5) & col('Genre').contains("Comedy") ).show()


In [ ]:
sdf.filter( (col('Genre').contains("Action")) | col('Genre').contains("Comedy") ).show()

In [ ]:
sdf_top = sdf.filter( (col("Rating") >= 5) & col('Genre').contains("Comedy") )
sdf_top.show()


#### Transform data

Transforming columns

In [ ]:
sdf.withColumn("Rating", sdf["Rating"] / 5 ).show(5)

In [ ]:
sdf.withColumnRenamed("Movie_Name", "Title").show(5)

#### Apply user defined functions

In [ ]:
import pandas as pd
from math import log
from pyspark.sql.functions import pandas_udf

@pandas_udf('float')
def pandas_log(series: pd.Series) -> pd.Series:
    return series.apply(log)

sdf.withColumn("log_rating", pandas_log(sdf["Rating"])).show(5)


In [ ]:
from pyspark.sql.types import ArrayType, StringType

@pandas_udf(ArrayType(StringType()))
def pandas_split(series: pd.Series) -> pd.Series:
    return series.apply(lambda x: x.split("|") if isinstance(x, str) else None)

sdf.withColumn("First_Genre", pandas_split(sdf["Genre"])).show(5)


#### Grouping data

To transform the data, we can use many of the functions available in the Spark DataFrame API. For example, we can use the `groupBy` function to group the data by the `Movie_Name` column and then use the `agg` function to apply the `count` function to each group.


In [ ]:
from pyspark.sql import functions as sf

sdf_top.groupBy("Movie_name").agg(
    sf.count("*").alias("total_ratings"),
).show()

Here, we filter dataset to only `Genre` that contains `Comedy`, group by `Movie_Name` and `Year`, and summarize the total ratings and average ratings.

In [ ]:
sdf.filter( sf.col('Genre').contains("Comedy") ). \
    groupBy("Movie_name").agg(
        sf.count("*").alias("total_ratings"),
        sf.avg("Rating").alias("avg_rating")
    ).show()

Add `sum` to summarize the `Ratting` column.

In [ ]:
sdf.filter( sf.col('Genre').contains("Comedy") ). \
    groupBy("Movie_name").agg(
        sf.count("*").alias("total_ratings"),
        sf.avg("Rating").alias("avg_rating"),
        sf.sum("Rating").alias("sum_rating")
    ). \
   orderBy("total_ratings", ascending=False).show()

#### Collect data from Spark to Pandas dataframe

After filter, groupBy, aggregate data, and order, we can collect data from Spark using `.toPandas()` method.

In [ ]:
from pyspark.sql import functions as sf

df = sdf.filter( sf.col('Genre').contains("Comedy") ). \
    groupBy("Movie_name").agg(
        sf.count("*").alias("total_ratings"),
        sf.avg("Rating").alias("avg_rating"),
        sf.sum("Rating").alias("sum_rating")
    ). \
    filter( sf.col("avg_rating") > 4). \
    orderBy("total_ratings", ascending=False). \
    toPandas()

df

Collect as list

In [ ]:
data_collected = sdf.filter( sf.col('Genre').contains("Comedy") ). \
    groupBy("Movie_name").agg(
        sf.count("*").alias("total_ratings"),
        sf.avg("Rating").alias("avg_rating"),
        sf.sum("Rating").alias("sum_rating")
    ). \
    filter( sf.col("avg_rating") > 4). \
    orderBy("total_ratings", ascending=False). \
    collect()
data_collected

In [ ]:
def plus_mean(pandas_df):
    return pandas_df.assign(Rating=pandas_df.Rating - pandas_df.Rating.mean())

result_df = sdf.groupby('User_Id').applyInPandas(plus_mean, schema=sdf.schema)
result_df.show()

In [ ]:
spark.stop()